In [13]:
import pandas as pd
import glob
import os

Juntar todos los años

In [14]:
# Ruta a la carpeta que contiene los archivos
ruta = 'Data/ExtraccionDomiVersionFinal/'

# Obtener todos los archivos que coincidan con el patrón
archivos = glob.glob(os.path.join(ruta, 'ListaCobroDetalle20*.csv'))

# Leer y concatenar todos los archivos en un solo DataFrame
df_detalle = pd.concat([pd.read_csv(archivo) for archivo in archivos], ignore_index=True)

# Verificar las primeras filas del DataFrame consolidado
# print(df_detalle.head())


C:\Users\conej\AppData\Local\Temp\ipykernel_12336\3647624057.py:8: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_detalle = pd.concat([pd.read_csv(archivo) for archivo in archivos], ignore_index=True)


In [15]:
# Cargar catálogos
cat_banco = pd.read_csv(os.path.join(ruta, 'CatBanco.csv'), sep='\t')
cat_emisora = pd.read_csv(os.path.join(ruta, 'CatEmisora.csv'), sep='\t')
# cat_respuesta = pd.read_csv(os.path.join(ruta, 'CatRespuestaBancos.csv'), sep='\t')
cat_respuesta = pd.read_csv('Data\ExtraccionDomiVersionFinal\CatRespuestaBancos.csv', sep=',', engine='python')


In [16]:
# Agrupar por idCredito y aplicar funciones personalizadas
estadisticas = df_detalle.groupby('idCredito').agg({
    'montoExigible': lambda x: x.unique().sum(),
    'montoCobrar':   lambda x: x.unique().sum(),
    'montoCobrado':  'sum',  # aquí sí queremos el total cobrado
    'idListaCobro':  'count',
}).rename(columns={'idListaCobro': 'num_cobros'})

# Verificar las estadísticas calculadas
print(estadisticas.head())


           montoExigible  montoCobrar  montoCobrado  num_cobros
idCredito                                                      
6                 375.47       375.47           0.0           2
560               572.44       572.44           0.0           1
773               391.12       391.12           0.0           2
1154               92.73        92.73           0.0           2
1315              375.47       375.47           0.0           2


In [17]:
# Unir con el catálogo de respuestas de bancos
df_detalle = df_detalle.merge(cat_respuesta, how='left', left_on='idRespuestaBanco', right_on='IdRespuestaBanco')

# Verificar las primeras filas después de la unión
print(df_detalle[['idCredito', 'idRespuestaBanco', 'Descripcion']].head())


   idCredito idRespuestaBanco                  Descripcion
0      83021               04  Cuenta Insuficiencia Fondos
1      87227               04  Cuenta Insuficiencia Fondos
2     103785               04  Cuenta Insuficiencia Fondos
3     110463               04  Cuenta Insuficiencia Fondos
4     113755               04  Cuenta Insuficiencia Fondos


Estadística solo con montos pagados

In [18]:
estadistica_final = estadisticas[estadisticas['montoCobrado'] > 0]

In [19]:
estadistica_final = estadisticas[estadisticas['montoCobrado'] > 0].copy()
estadistica_final['porcentaje_pagado'] = (
    estadistica_final['montoCobrado'] / estadistica_final['montoExigible']
) * 100

In [20]:
print(estadistica_final[['montoExigible', 'montoCobrado', 'porcentaje_pagado']].head())


           montoExigible  montoCobrado  porcentaje_pagado
idCredito                                                
4635              613.82        480.39          78.262357
4914              969.96        384.98          39.690297
10983            3009.51       6019.02         200.000000
17740             529.93       1589.79         300.000000
18720             433.09        433.09         100.000000


In [21]:
print(estadistica_final['montoExigible'].sum())
print(estadistica_final['montoCobrado'].sum())
print(estadistica_final['porcentaje_pagado'].mean())

99704761.66
306253237.55
368.05646281502464


In [22]:
conteo_respuestas = df_detalle['idRespuestaBanco'].value_counts()
print(conteo_respuestas)


idRespuestaBanco
4.0        8980842
04          493339
0.0         275317
DD00031      40128
13.0         33864
00           18586
26.0         14859
2.0           9090
8.0           8374
3.0           6175
1.0           5087
08            4973
DD00020       3411
26            2290
DD00028       1824
DD00021       1542
30.0          1473
36.0          1343
13            1149
03             960
02             865
30             579
01             358
40.0           312
INC9           292
INC16          267
19.0           205
DD00041         87
DD00033         82
88.0            64
42.0            37
10.0            29
19              23
10               8
88               7
84.0             7
DD00042          6
40               5
42               5
36               5
18.0             5
6.0              2
53               1
83               1
37.0             1
Name: count, dtype: int64


In [24]:
print(cat_respuesta['IdRespuestaBanco'].value_counts())

IdRespuestaBanco
00         1
DD00025    1
DOM5       1
DOM4       1
DOM3       1
          ..
83         1
82         1
81         1
53         1
DD00043    1
Name: count, Length: 105, dtype: int64
